In [ ]:
import os
import tifffile as tiff
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.measure import label,regionprops, regionprops_table
from skimage.measure import regionprops
from skimage.util import map_array
import numpy as np
import rasterio
from rasterio.features import geometry_mask
import rasterio.mask
import rasterstats 
from rasterio.mask import mask

In [ ]:
# pip install rasterio
#pip install rasterstats

In [ ]:
#read the data
def read_data(rgb_path, nir_path,parcel_path, building_path):
    # Read the raster data
    with rasterio.open(rgb_path) as src:
        rgb = src.read()  # Read all bands (rows, columns)
        rgb = np.transpose(rgb, (1, 2, 0)) # Change the order of the bands to (rows, columns, bands)


    with rasterio.open(nir_path) as src:
        nir = src.read(1) 

    # Read the vector data
    parcels = gpd.read_file(parcel_path)
    buildings = gpd.read_file(building_path)
    
    #ensure the data have the same coordinate system
    buildings = buildings.to_crs(src.crs)
    parcels = parcels.to_crs(src.crs)

    return rgb, nir, parcels, buildings,src

In [ ]:
rgb_path='/kaggle/input/ubicube-work-sample/aerial/AOI_ID_1_ortho_cog_RGB.tif'
nir_path='/kaggle/input/ubicube-work-sample/aerial/AOI_ID_1_ortho_cog_NIR.tif'
bld_path='/kaggle/input/ubicube-work-sample/building_footprints/building_footprints.geojson'
par_path='/kaggle/input/ubicube-work-sample/parcels/parcels.shp'

rgb, nir, parcels, buildings,src = read_data(rgb_path, nir_path,par_path, bld_path)

In [ ]:
plt.imshow(rgb)

In [ ]:
plt.imshow(nir)

In [ ]:
# Extract the bands
red = rgb_transposed[:, :, 0]
green = rgb_transposed[:, :, 1]
blue = rgb_transposed[:, :, 2]

In [ ]:
plt.imshow(blue)

In [ ]:
# Normalize the bands
red_norm = (red - np.min(red)) / (np.max(red) - np.min(red))
green_norm = (green - np.min(green)) / (np.max(green) - np.min(green))
blue_norm = (blue - np.min(blue)) / (np.max(blue) - np.min(blue))
nir_norm =  (nir - np.min(nir)) / (np.max(nir) - np.min(nir))

In [ ]:
#compute ndvi and adjust for scenarios where you might have to divide by 0
denominator_zero_mask = (nir_norm + red_norm) == 0
ndvi = np.where(denominator_zero_mask, 0, (nir_norm - red_norm) / (nir_norm + red_norm))

In [ ]:
plt.imshow(ndvi)

In [ ]:
#object based image segmentation, using the SLIC algorithm
def image_segmentation(image, algorithm='slic', algorithm_params=None, channel_axis=-1):
    if algorithm_params is None:
        algorithm_params = {}

    # Perform segmentation
    segments_result = slic(image, **algorithm_params, channel_axis=channel_axis)
    
    # Mark boundaries on the original image using the segmented result
    boundaries = mark_boundaries(image[:, :, [0, 1, 2]], segments_result, (1,0,0), mode="thick")

    return segments_result, boundaries

In [ ]:
#Function to plot the segments

def plot_segments(image,segments,boundaries):
        # Display the original image and the segmentation result
    fig, ax = plt.subplots(1, 3, figsize=(12, 6))

    ax[0].imshow(image[:,:,[0,1,2]])
    ax[0].set_title(f'Tile')

    ax[1].imshow(segments)
    ax[1].set_title(f'Segments_Tile')

    ax[2].imshow(boundaries)
    ax[2].set_title(f'Segments with boundaries')

    plt.show()

In [ ]:
#segment the image: testing various parameter values
slic_params = {'n_segments': 1000, 'compactness': 0.5}
algorithm='slic'
segments, boundaries = image_segmentation(image1, algorithm=algorithm, algorithm_params=slic_params)

In [ ]:
plot_segments(image1,segments, boundaries)

In [ ]:


# # Calculate ndvi values for each segment
# segment_props = regionprops(segments, intensity_image=ndvi)

# # Extract the mean ndvi values for each labeled segment
# segment_features = []
# for prop in segment_props:
#     segment_features.append({
#         'label': prop.label,
#         'mean_intensity': prop.mean_intensity,
#         'area': prop.area
       
#     })

In [ ]:

# Calculate ndvi values for each segment and output as a dataframe
def calc_segment_ndvi(seg_arr, img_arr):
    spec_feats = regionprops_table(
        label_image = seg_arr,
        intensity_image = img_arr,
        properties = ["label", "intensity_mean"]
        )
    return pd.DataFrame(spec_feats)

In [ ]:
seg_ndvi= calc_segment_ndvi(segments,ndvi)

In [ ]:
seg_ndvi

# Option 2

In [ ]:
#Map the ndvi values back to the image segments
mapped_ndvi=map_array(segments,np.array(seg_ndvi['label']),np.array(seg_ndvi['intensity_mean']))

# Define NDVI range for vegetated areas
ndvi_min =-1
ndvi_max =0.1

# Create a masked NDVI array, these are the impervious surface
impervious_surfaces = np.where((mapped_ndvi >= ndvi_min) & (mapped_ndvi < ndvi_max), mapped_ndvi, np.nan)

In [ ]:
#plot the resultant images
fig, axs = plt.subplots(ncols=4, figsize=(20,10), constrained_layout=True)
axs[0].imshow(rgb)
axs[1].imshow(boundaries)
axs[3].imshow(mapped_ndvi, cmap="RdYlGn")
axs[2].imshow(impervious_surfaces, cmap='YlGn')

In [ ]:

#create a building mask where pixels within the building are 0 and the rest 1
building_mask = geometry_mask([geom for geom in buildings.geometry], transform=src.transform, out_shape=image.shape[1:])

# Mask the impervious surfaces array with the building mask
impervious_surfaces_no_buildings = np.where(building_mask, impervious_surfaces, np.nan)

# Display the result
fig, axs = plt.subplots(ncols=4, figsize=(20,10), constrained_layout=True)
axs[0].imshow(image1)
axs[1].imshow(boundaries)
axs[2].imshow(impervious_surfaces_no_buildings, cmap='YlGn')
axs[3].imshow(building_mask)

In [ ]:

impervious_mask = impervious_surfaces > 0  # Binary mask for impervious areas

# Calculate zonal statistics - sum of impervious pixels within each parcel polygon
stats = rasterstats.zonal_stats(parcels, impervious_surfaces_no_buildings, affine=src.transform, stats="sum", nodata=src.nodata)

# Add the impervious area (sum of impervious pixels) to the GeoDataFrame
parcels["impervious_area_pixels"] = [s['sum'] if s['sum'] is not None else 0 for s in stats]
parcels["impervious_area"] = parcels["impervious_area_pixels"]*0.4 #to compute area in real world units(m2) by multiplying by a pixel area (0.2m resolution)

# Calculate impervious area percentage
parcels["unsealing_potential"] = (parcels["impervious_area"] / parcels.area) * 100

In [ ]:
parcels

In [ ]:
# Plot the parcels showing unsealing potential
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
parcels.plot(column="impervious_area", cmap="OrRd", legend=True, ax=ax)
plt.title("Unsealing Potential of Parcels")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

# Save the updated GeoDataFrame with unsealing potential to a new shapefile
output_path = "/kaggle/working/parcels_unsealing_potential.shp"
parcels.to_file(output_path)

print(f"Shapefile with unsealing potential saved to {output_path}")